In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from aust_covid.inputs import load_google_mob_year_df, load_raw_pop_data
PROJECT_PATH = Path().resolve().parent
DATA_PATH = PROJECT_PATH / 'data'

In [ ]:
locations = [
    'school', 
    'home', 
    'work', 
    'other_locations',
]

In [ ]:
raw_location_matrices = {i: pd.read_csv(DATA_PATH / f'{i}.csv', index_col=0).to_numpy() for i in locations}

In [ ]:
state_averages

In [ ]:
raw_data_2021 = pd.read_csv(DATA_PATH / '2021_AU_Region_Mobility_Report.csv', index_col=8)
raw_data_2022 = pd.read_csv(DATA_PATH / '2022_AU_Region_Mobility_Report.csv', index_col=8)
raw_data = pd.concat([raw_data_2021, raw_data_2022])
state_data = raw_data.loc[raw_data['sub_region_1'].notnull() & raw_data['sub_region_2'].isnull()]
wa_data = state_data.loc[state_data['sub_region_1'] == 'Western Australia']
non_wa_data = state_data.loc[state_data['sub_region_1'] != 'Western Australia']
mob_cols = [c for c in wa_data.columns if '_percent_change_from_baseline' in c]
wa_mobility = wa_data[mob_cols]
mob_locations = [c.replace('_percent_change_from_baseline', '') for c in wa_mobility.columns]
wa_mobility.columns = mob_locations
non_wa_mobility = non_wa_data[['sub_region_1'] + mob_cols]
non_wa_mobility.columns = ['sub_region_1'] + mob_locations
wa_mobility.index = pd.to_datetime(wa_mobility.index)
non_wa_mobility.index = pd.to_datetime(non_wa_mobility.index)
state_pop_totals = load_raw_pop_data('31010do002_202206.xlsx').sum()
for jurisdiction in state_pop_totals.index:
    non_wa_mobility.loc[non_wa_mobility['sub_region_1'] == jurisdiction, 'weights'] = state_pop_totals[jurisdiction]
state_averages = pd.DataFrame(columns=mob_locations)
for location in mob_locations:
    location_mob = non_wa_mobility.groupby(non_wa_mobility.index).apply(lambda x: np.average(x[location], weights=x['weights']))
    state_averages[location] = location_mob
non_wa_relmob = 1.0 + state_averages * 1e-2
wa_relmob = 1.0 + wa_mobility * 1e-2

In [ ]:
state_pop_totals = load_raw_pop_data('31010do002_202206.xlsx').sum()

In [ ]:
state_pop_totals.index

In [ ]:
for jurisdiction in state_pop_totals.index:
    non_wa_mobility.loc[non_wa_mobility['sub_region_1'] == jurisdiction, 'fake_weights'] = state_pop_totals[jurisdiction]